# D07 Biến đổi dữ liệu (transformation): `apply()`

## Mục đích

Giới thiệu hàm `apply()` và `applymap()` để biến đổi dữ liệu trong Pandas.


## Các cách thức biến đổi dữ liệu trong Pandas

Trên thực tế, chúng ta đã tìm hiểu một số cách thức biến đổi dữ liệu trong Pandas. Chẳng hạn, sử dụng biểu thức số học hoặc logic để tạo ra biến mới.

In [1]:
import pandas as pd
import numpy as np

d = pd.read_excel("hrm.xlsx", index_col="id")
d["date_exam"].dt.year - d["yob"]

id
223     44
236     48
256     49
296     37
310     39
        ..
4200    36
4214    47
4216    58
4220    57
4240    35
Length: 330, dtype: int64

Hoặc sử dụng các hàm masking (`mask()` hoặc `where()`).

In [2]:
d["eso_LA"].gt(0).mask(d["eso_LA"].isna())

id
223      True
236      True
256      True
296      True
310     False
        ...  
4200      NaN
4214    False
4216    False
4220    False
4240    False
Name: eso_LA, Length: 330, dtype: object

Hoặc sử dụng các hàm thay thế (`replace()` và `map()`).

In [3]:
d["sex"].replace({0: "Nữ", 1: "Nam"})

id
223     Nam
236     Nam
256     Nam
296     Nam
310      Nữ
       ... 
4200     Nữ
4214     Nữ
4216     Nữ
4220     Nữ
4240     Nữ
Name: sex, Length: 330, dtype: object

Nhưng nếu phép biến đổi (transformation) mà bạn sử dụng không có sẵn trong Pandas, chúng ta sẽ phải viết một hàm riêng cho nó. Chẳng hạn, nếu muốn tạo một biến kết quả *H. pylori* dựa trên điều kiện như sau:

* Nếu có một trong hai kết quả xét nghiệm `"hp_endo"` hoặc `"hp_breath"` thì sử dụng kết quả đó.
* Nếu có cả hai kết quả thì xét nghiệm chỉ âm tính khi cả hai kết quả đều âm tính.

Chúng ta có thể tạo ra một biểu thức logic cho việc này, nhưng sẽ hơi mất thời gian. Chúng ta có thể làm đơn giản hơn bằng hàm sau.

In [4]:
def hp_result(hp_endo, hp_breath):
    # Nếu cả hai đều là NA, trả về NA
    if pd.isna(hp_endo) and pd.isna(hp_breath):
        return np.nan
    
    # Nếu cả hai đều không phải NA,
    # trả về False khi cả hai đều bằng 0
    if not (pd.isna(hp_endo) or pd.isna(hp_breath)):
        return (hp_endo == 1) | (hp_breath == 1)
    
    # Nếu một trong hai không phải NA,
    # trả về kết quả không phải NA
    return (hp_endo == 1) if pd.isna(hp_breath) else (hp_breath == 1)

Bạn có thể test thử như dưới đây. Câu hỏi đặt ra là làm sao chúng ta áp dụng hàm này cho toàn bộ các bản ghi trong data frame.

In [5]:
print(
    hp_result(0, 0),
    hp_result(0, None),
    hp_result(1, 0),
    hp_result(None, 1)
)

False False True True


## Hàm `apply()`

Tương tự hàm `map()` có sẵn trong Python hay hàm `apply()` trong thư viện Numpy, Pandas cũng hỗ trợ việc lặp thông qua hàm `apply()`. Bạn có thể dùng hàm `apply()` cho data frame hoặc series, nhưng sẽ hơi khác nhau một chút. Chúng ta sẽ tiếp tục ví dụ trên trước.

In [6]:
d[["hp_endo", "hp_breath"]].apply(lambda x: hp_result(*x), axis=1)

id
223     False
236     False
256      True
296      True
310     False
        ...  
4200      NaN
4214     True
4216    False
4220     True
4240     True
Length: 330, dtype: object

Bạn sẽ thấy một vài chi tiết kĩ thuật trong hàm `apply()`:

* Đối số đầu tiên là *hàm sẽ lặp qua các hàng hoặc cột*. Nếu hàm của bạn chỉ nhận một đối số duy nhất, bạn có thể chỉ cần cung cấp tên hàm là đủ. Trong trường hợp của chúng ta, hàm `hp_result()` có hai đối số, do đó bạn sẽ cần đến hàm lambda để gọi được hàm này. Chúng ta dùng kĩ thuật unpacking với một dấu sao để giải nén nội dung của từng hàng (là một tuple bao gồm hai giá trị của cột `"hp_endo"` và `"hp_breath"`).
* Đối số thứ hai là `axis`, quy định chiều không gian chúng ta sẽ thao tác. Ở đây, chúng ta thiết lập `axis=1` (cột) vì chúng ta muốn lặp theo hàng, mỗi lần lặp sẽ trả về thông tin của các cột của từng hàng.

Bạn cũng có thể lặp trên series. Hàm `apply()` của series không có đối số `axis` (vì chúng ta không cần đến nó).

In [7]:
d["sex"].apply(lambda x: "Nam" if x == 1 else "Nữ")

id
223     Nam
236     Nam
256     Nam
296     Nam
310      Nữ
       ... 
4200     Nữ
4214     Nữ
4216     Nữ
4220     Nữ
4240     Nữ
Name: sex, Length: 330, dtype: object

### Kết quả trả về của hàm là danh sách

Trong một số trường hợp, hàm của chúng ta sẽ trả về một danh sách (hoặc tuple, set, v.v.). Ví dụ bạn viết một hàm để tính cả BMI và BSA của bệnh nhân (BSA theo Haycock được tính như sau: $\text{BSA} (m^2) = w (kg)^{0.5378} \times h (cm)^{0.3964} \times 0.024265$).

In [8]:
def body_indices(w, h):
    if pd.isna(w) or pd.isna(h):
        return np.nan, np.nan
    
    return w / h ** 2, w ** 0.5378 * (h * 100) ** 0.3964 * 0.024265

body_indices(60, 1.65)

(22.03856749311295, 1.6606598033454585)

Nếu dùng hàm `apply()` như ở trên, chúng ta sẽ có kết quả như sau:

In [9]:
d[["weight", "height"]].apply(lambda x: body_indices(*x), axis=1)

id
223     (22.758306781975424, 1.7502471879547865)
236     (23.306680053067517, 1.7420070489068296)
256       (20.41522491349481, 1.665307584246721)
296     (23.938989774631512, 1.8299266404185566)
310      (19.976218787158146, 1.302356974386245)
                          ...                   
4200    (19.067710657633167, 1.4278715677393228)
4214     (23.53036634346221, 1.5988454774446972)
4216     (19.024970273483948, 1.268628416419575)
4220     (20.66115702479339, 1.4491086913307503)
4240    (22.582709172343712, 1.6526512045433615)
Length: 330, dtype: object

Đây không phải là kết quả chúng ta mong muốn. Chúng ta muốn trả về hai cột. Bạn có thể thêm đối số `result_type` như sau.

In [10]:
d[["weight", "height"]].apply(lambda x: body_indices(*x), axis=1, result_type="expand")

,0,1
id,,
223,22.758307,1.750247
236,23.306680,1.742007
256,20.415225,1.665308
296,23.938990,1.829927
310,19.976219,1.302357
...,...,...
4200,19.067711,1.427872
4214,23.530366,1.598845
4216,19.024970,1.268628


## Hàm `applymap()`

Trong trường hợp muốn áp dụng cùng một hàm cho tất cả các hàng ở nhiều cột khác nhau, chúng ta sử dụng hàm `applymap()`. Ví dụ:

In [11]:
d[["hp_endo", "hp_breath"]].applymap(lambda x: "POS" if x == 1 else "NEG / NA")

,hp_endo,hp_breath
id,,
223,NEG / NA,NEG / NA
236,NEG / NA,NEG / NA
256,NEG / NA,POS
296,POS,NEG / NA
310,NEG / NA,NEG / NA
...,...,...
4200,NEG / NA,NEG / NA
4214,POS,POS
4216,NEG / NA,NEG / NA


Các hàm `apply()` và `applymap()` đều được vector hóa trong Pandas, nên tốc độ xử lí cao và ổn định hơn việc lặp qua từng bản ghi bằng vòng lặp. Tuy nhiên, nếu có thể sử dụng các hàm sẵn có của Pandas, tốc độ xử lí thường sẽ nhanh hơn. Trong thực tế, đa số trường hợp bạn sẽ không cần dùng đến hàm `apply()`, nhưng trong một số tình huống nhất định bạn sẽ cần đến nó, hãy xem ở bài sau nhé.

---

[Bài trước](./06_groupby.ipynb) - [Danh sách bài](../README.md) - [Bài sau](./08_agg.ipynb)